# System for access segmentation

##### imports

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.cluster import AgglomerativeClustering

#### Get data from results/data folder

In [2]:
data = pd.data = pd.read_excel("../server/public/data/User-624921c5e1c4d602ddd19695-1649816982449.xlsx", skiprows=1)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# data = pd.read_excel("/content/drive/MyDrive/User-624921c5e1c4d602ddd19695-1649206670127.xlsx", skiprows=1)

In [5]:
data

,0000081,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CO3180_DP:D_CO_DEFAULT
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO_DEFAULT
...,...,...,...,...
221273,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER
221274,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE
221275,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...
221276,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD


In [6]:
data.columns = ["user_id", "job", "department", "privileges_data"]

In [7]:
data = data.dropna()

In [8]:
class_to_counts = data.privileges_data.value_counts()
class_to_counts

PRIV:ROLE:PE1CLNT110:XX0000_SP:D_EVERYONE          2278
PRIV:ROLE:PGWCLNT110:XX0000_SP:D_EVERYONE          2115
PRIV:ROLE:PGWCLNT110:XX0000_SP:D_EVERYONE_FIORI    2115
PRIV:ROLE:QE2CLNT110:XX0000_SP:D_EVERYONE          2110
PRIV:ROLE:PO1CLNT110:SM0000_SP:D_EVERYONE_PO1      2049
                                                   ... 
PRIV:ROLE:P11CLNT110:CO0980_DP:D_CO_REP_GIDE          1
PRIV:ROLE:QE2CLNT110:FI4100_DP:M_TRAVEL_FI            1
PRIV:ROLE:QE2CLNT110:FI3220_DP:M_POST_CUSTOMER        1
PRIV:ROLE:QE2CLNT110:CS3240_DP:M_CS_ORDER             1
PRIV:ROLE:QG2CLNT110:GTS1500_DP:M_LC                  1
Name: privileges_data, Length: 8495, dtype: int64

In [9]:
data['times_occured'] = data['privileges_data'].apply(lambda y: class_to_counts[y])

In [10]:
data.times_occured = data.times_occured.astype(int)

In [11]:
data.dtypes

user_id             int64
job                object
department         object
privileges_data    object
times_occured       int64
dtype: object

In [12]:
data

,user_id,job,department,privileges_data,times_occured
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT,72
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC,72
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT,72
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO,72
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO_DEFAULT,72
...,...,...,...,...,...
221273,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER,9
221274,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE,90
221275,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...,104
221276,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD,108


In [13]:
data_with_privileges_found_one_time = data.where(data['times_occured'] == 1).dropna()

In [14]:
data_with_privileges_found_one_time.dtypes

user_id            float64
job                 object
department          object
privileges_data     object
times_occured      float64
dtype: object

In [15]:
data_with_privileges_found_one_time.times_occured = data_with_privileges_found_one_time.times_occured.astype(int)

In [16]:
data_with_privileges_found_one_time.dtypes

user_id            float64
job                 object
department          object
privileges_data     object
times_occured        int64
dtype: object

In [17]:
data_with_privileges_found_one_time

,user_id,job,department,privileges_data,times_occured
1143,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:MM3180_DP:M_REQUISITION_REL,1
1147,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_KEYUSER,1
1148,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_MD_CUSTOMER,1
1149,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_MD_SALES_PRICE,1
1151,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_SD,1
...,...,...,...,...,...
220110,4023508.0,FIELD SERVICE,OPÉRATEUR DE SAISIE,PRIV:ROLE:PE1CLNT110:PS3270_DP:M_PROJECT_MANAGER,1
220891,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3230_DP:M_CATS_ADMIN,1
220893,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3230_DP:M_CATS_X,1
220894,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3250_DP:M_CATS_ADMIN,1


In [18]:
data.drop(data_with_privileges_found_one_time.index, inplace=True)

In [19]:
data = data.reset_index(drop=True)
data

,user_id,job,department,privileges_data,times_occured
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT,72
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC,72
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT,72
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO,72
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO_DEFAULT,72
...,...,...,...,...,...
219529,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER,9
219530,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE,90
219531,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...,104
219532,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD,108


In [20]:
data.drop('times_occured', axis=1, inplace=True)

In [21]:
X_train, y_train = train_test_split(data, train_size=0.4, shuffle=True, stratify=data.privileges_data)

In [22]:
X_train

,user_id,job,department,privileges_data
69665,2187682,FIELD SERVICE,SERVICETECHNIKER ACT,PRIV:ROLE:QE2CLNT110:SD3030_DP:D_SD_DEFAULT
125208,4004067,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:MM4100_DP:D_PURCHASING
30673,522300,FIELD SERVICE,SERVICE & TEST ENGINEER NORTH WEST,PRIV:ROLE:QE2CLNT110:SD3120_DP:D_SD_DEFAULT
147699,4013020,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:PP4100_DP:D_PP
7877,1482,FIELD SERVICE,"SERVICE, SERVICE IC",PRIV:ROLE:PE1CLNT110:LO8030_DP:D_LO_DEFAULT
...,...,...,...,...
210824,4022588,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3010_DP:M_CATS
198060,4021043,SALES,SALES CONSULTANT SWP+IC,PRIV:ROLE:QE2CLNT110:CS3040_DP:D_CS_STATISTIC
209139,4022456,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:CS3270_DP:M_MD
127530,4004846,FIELD SERVICE,FIELD SERVICE TECHNICIAN,PRIV:ROLE:PO1CLNT110:XX0000_SP:D_EVERYONE


In [23]:
data = X_train

In [24]:
data.shape

(87813, 4)

In [25]:
data

,user_id,job,department,privileges_data
69665,2187682,FIELD SERVICE,SERVICETECHNIKER ACT,PRIV:ROLE:QE2CLNT110:SD3030_DP:D_SD_DEFAULT
125208,4004067,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:MM4100_DP:D_PURCHASING
30673,522300,FIELD SERVICE,SERVICE & TEST ENGINEER NORTH WEST,PRIV:ROLE:QE2CLNT110:SD3120_DP:D_SD_DEFAULT
147699,4013020,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:PP4100_DP:D_PP
7877,1482,FIELD SERVICE,"SERVICE, SERVICE IC",PRIV:ROLE:PE1CLNT110:LO8030_DP:D_LO_DEFAULT
...,...,...,...,...
210824,4022588,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3010_DP:M_CATS
198060,4021043,SALES,SALES CONSULTANT SWP+IC,PRIV:ROLE:QE2CLNT110:CS3040_DP:D_CS_STATISTIC
209139,4022456,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:CS3270_DP:M_MD
127530,4004846,FIELD SERVICE,FIELD SERVICE TECHNICIAN,PRIV:ROLE:PO1CLNT110:XX0000_SP:D_EVERYONE


In [26]:
#data = data.append(data_with_privileges_found_one_time.drop('times_occured', axis=1))

In [27]:
data = data.reset_index(drop=True)

In [28]:
data.shape

(87813, 4)

In [29]:
data

,user_id,job,department,privileges_data
0,2187682,FIELD SERVICE,SERVICETECHNIKER ACT,PRIV:ROLE:QE2CLNT110:SD3030_DP:D_SD_DEFAULT
1,4004067,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:MM4100_DP:D_PURCHASING
2,522300,FIELD SERVICE,SERVICE & TEST ENGINEER NORTH WEST,PRIV:ROLE:QE2CLNT110:SD3120_DP:D_SD_DEFAULT
3,4013020,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN,PRIV:ROLE:QE2CLNT110:PP4100_DP:D_PP
4,1482,FIELD SERVICE,"SERVICE, SERVICE IC",PRIV:ROLE:PE1CLNT110:LO8030_DP:D_LO_DEFAULT
...,...,...,...,...
87808,4022588,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3010_DP:M_CATS
87809,4021043,SALES,SALES CONSULTANT SWP+IC,PRIV:ROLE:QE2CLNT110:CS3040_DP:D_CS_STATISTIC
87810,4022456,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:CS3270_DP:M_MD
87811,4004846,FIELD SERVICE,FIELD SERVICE TECHNICIAN,PRIV:ROLE:PO1CLNT110:XX0000_SP:D_EVERYONE


In [30]:
user_data = data.drop("privileges_data", axis=1)

In [31]:
user_data

,user_id,job,department
0,2187682,FIELD SERVICE,SERVICETECHNIKER ACT
1,4004067,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN
2,522300,FIELD SERVICE,SERVICE & TEST ENGINEER NORTH WEST
3,4013020,FIELD SERVICE,SR. FIELD SERVICE TECHNICIAN
4,1482,FIELD SERVICE,"SERVICE, SERVICE IC"
...,...,...,...
87808,4022588,CUSTOMER SERVICE,JUNIOR SPECIALIST CS
87809,4021043,SALES,SALES CONSULTANT SWP+IC
87810,4022456,CUSTOMER SERVICE,SPECIALIST CS
87811,4004846,FIELD SERVICE,FIELD SERVICE TECHNICIAN


In [32]:
privileges_data = pd.DataFrame(data["privileges_data"])

In [33]:
privileges_data.privileges_data = privileges_data["privileges_data"].str[10:]

I saw anomaly in data where in some of the records there is ::(double) symbols so i need to remove it so the data is clear

In [34]:
for indx in privileges_data[privileges_data["privileges_data"].str.startswith(":")].index.tolist():
    privileges_data.loc[indx].privileges_data = privileges_data.loc[indx].privileges_data[1:]

In [35]:
len(privileges_data.loc[privileges_data["privileges_data"].str.startswith(":")].privileges_data)

0

In [36]:
privileges_data

,privileges_data
0,QE2CLNT110:SD3030_DP:D_SD_DEFAULT
1,QE2CLNT110:MM4100_DP:D_PURCHASING
2,QE2CLNT110:SD3120_DP:D_SD_DEFAULT
3,QE2CLNT110:PP4100_DP:D_PP
4,PE1CLNT110:LO8030_DP:D_LO_DEFAULT
...,...
87808,PE1CLNT110:XX3010_DP:M_CATS
87809,QE2CLNT110:CS3040_DP:D_CS_STATISTIC
87810,PE1CLNT110:CS3270_DP:M_MD
87811,PO1CLNT110:XX0000_SP:D_EVERYONE


In [37]:
privileges_data['priv'] = privileges_data['privileges_data']
privileges_data['system_name'] = privileges_data['privileges_data'].str.split(':', 1, expand=True)[0]

In [38]:
privileges_data = privileges_data.drop('privileges_data', axis=1)

In [39]:
privileges_data

,priv,system_name
0,QE2CLNT110:SD3030_DP:D_SD_DEFAULT,QE2CLNT110
1,QE2CLNT110:MM4100_DP:D_PURCHASING,QE2CLNT110
2,QE2CLNT110:SD3120_DP:D_SD_DEFAULT,QE2CLNT110
3,QE2CLNT110:PP4100_DP:D_PP,QE2CLNT110
4,PE1CLNT110:LO8030_DP:D_LO_DEFAULT,PE1CLNT110
...,...,...
87808,PE1CLNT110:XX3010_DP:M_CATS,PE1CLNT110
87809,QE2CLNT110:CS3040_DP:D_CS_STATISTIC,QE2CLNT110
87810,PE1CLNT110:CS3270_DP:M_MD,PE1CLNT110
87811,PO1CLNT110:XX0000_SP:D_EVERYONE,PO1CLNT110


In [40]:
user_data['system_name'] = privileges_data['system_name']
user_data['priv'] = privileges_data['priv']

In [41]:
job="POSTDOCTORAL FELLOW"
department="BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"

### Get information about dataset properties

#### Get columns

In [42]:
user_data.columns

Index(['user_id', 'job', 'department', 'system_name', 'priv'], dtype='object')

#### Get unique attributes in every column

In [43]:
for c in user_data.columns:
    print(c, len(user_data[c].value_counts()))

user_id 2343
job 3
department 726
system_name 20
priv 6751


#### Get unique privileges count for every system 

In [44]:
system_privs_count = 0
for s in user_data.system_name.unique():
    print(s, len(user_data[user_data.system_name == s].priv.unique()))
    system_privs_count += len(user_data[user_data.system_name == s].priv.unique())

print("Uniqua privs for every system count = ", system_privs_count)

QE2CLNT110 2504
PE1CLNT110 2182
DE2CLNT210 1584
PO1CLNT110 10
PGWCLNT110 10
QGWCLNT110 6
DGWCLNT110 5
PR1CLNT110 13
Q12CLNT110 103
P11CLNT110 204
QG2CLNT110 27
PB1CLNT110 23
PGRCLNT110 2
PG1CLNT110 33
QB1CLNT110 22
DG2CLNT210 14
SE6CLNT110 2
ASJAVAIDM 2
Q13CLNT110 2
PSRCLNT110 3
Uniqua privs for every system count =  6751


#### get how many times system occurs in data

In [45]:
user_data.system_name.value_counts()

QE2CLNT110    38227
PE1CLNT110    38040
DE2CLNT210     4283
PGWCLNT110     3217
PO1CLNT110     1911
P11CLNT110      968
PR1CLNT110      369
QGWCLNT110      218
Q12CLNT110      204
PG1CLNT110       68
DG2CLNT210       67
PB1CLNT110       54
DGWCLNT110       52
QG2CLNT110       50
QB1CLNT110       48
PGRCLNT110       16
ASJAVAIDM         8
PSRCLNT110        6
SE6CLNT110        5
Q13CLNT110        2
Name: system_name, dtype: int64

We can conclude that BCMP970 and IT systems contains special privileges for just one certain department and job

#### get data about privileges. Are there privileges which are the same for various systems

In [46]:
system_privs_count - len(user_data.priv.unique())

0

### Prepare data for clustering

In simple words we don't need data for job and department to cluster data, because clustering is performed on privileges for all users.

So we will create new table with user id - user_id, and all of the privileges as attributes with values of 0 or 1. 0 - don't have that role, 1 - have that role.

In [47]:
user_privs_data = pd.DataFrame()

Create all privileges(categorical data) compatabel for clustering by transforming all privs into numerical data.

In [48]:
priv_dummies = pd.DataFrame()

In [49]:
priv_dummies['user_id'] = user_data.user_id

In [50]:
priv_dummies = pd.concat([priv_dummies, pd.get_dummies(user_data.priv, dtype=np.ubyte)], axis=1)

In [51]:
priv_dummies.dtypes

user_id                                 int64
ASJAVAIDM:IDM_User                      uint8
ASJAVAIDM:idm.user                      uint8
DE2CLNT210:BC0000_DP:M_LOCUSER_DE       uint8
DE2CLNT210:CO3020_DP:D_CO               uint8
                                        ...  
QGWCLNT110:XX0000_SP:M_FIORI_CS         uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV        uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP    uint8
SE6CLNT110:XX0000_SP:D_EVERYONE         uint8
SE6CLNT110:ZSAP_ALL_ERP                 uint8
Length: 6752, dtype: object

In [52]:
priv_dummies = priv_dummies.drop('user_id', axis=1).astype('byte')

In [53]:
priv_dummies.dtypes

ASJAVAIDM:IDM_User                      int8
ASJAVAIDM:idm.user                      int8
DE2CLNT210:BC0000_DP:M_LOCUSER_DE       int8
DE2CLNT210:CO3020_DP:D_CO               int8
DE2CLNT210:CO3020_DP:M_CO_DEPT          int8
                                        ... 
QGWCLNT110:XX0000_SP:M_FIORI_CS         int8
QGWCLNT110:XX0000_SP:M_FIORI_TRV        int8
QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP    int8
SE6CLNT110:XX0000_SP:D_EVERYONE         int8
SE6CLNT110:ZSAP_ALL_ERP                 int8
Length: 6751, dtype: object

In [54]:
priv_dummies['user_id'] = user_data.user_id

In [ ]:
user_privs_data = priv_dummies.groupby('user_id').sum().reset_index(drop=True)

In [ ]:
user_privs_data['user_id'] = user_data.user_id

In [ ]:
agg_clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=15)

In [ ]:
fit_data = agg_clustering.fit(user_privs_data.drop('user_id', axis=1))

In [ ]:
fit_data.n_clusters_

In [ ]:
fit_data.n_features_in_

In [ ]:
fit_data.labels_

In [ ]:
result = pd.DataFrame(fit_data.labels_, columns=['id'])

In [ ]:
result.id.value_counts()

### Get result based on clustered data


#### Prepare data for extracting privileges from given label

In [ ]:
user_data

In [ ]:
unique_user_data = user_data.groupby(['user_id', 'job', 'department']).describe().reset_index()

In [ ]:
unique_user_data.columns

In [ ]:
unique_user_data

In [ ]:
unique_user_data.drop(['system_name', 'priv'], axis=1, inplace=True)

In [ ]:
unique_user_data.columns = ['user_id', 'job', 'department']

In [ ]:
unique_user_data

In [ ]:
result_table = pd.concat([unique_user_data, user_privs_data.drop('user_id', axis=1)], axis=1)

In [ ]:
result_table

In [ ]:
result_table.insert(3, "cluster_label", result.id)

In [ ]:
result_table

### Extract all data from clusters with selected job and department in them

In [ ]:
result_data = result_table.where((result_table.job == job) & (result_table.department == department))

In [ ]:
result_data.dropna(inplace=True)

In [ ]:
result_data = result_data.reset_index(drop=True)

In [ ]:
result_data

In [ ]:
result_data.cluster_label.value_counts()

#### Get  the best fitted cluster

In [ ]:
index = 1
# if len(result_data.cluster_label.value_counts().index.tolist()) > 3:
#   index = 2

best_cluster_label = result_data.cluster_label.value_counts().index.tolist()[0: index]

In [ ]:
best_cluster_label

In [ ]:
privileges_counts = {}
users_count = 0
for c in best_cluster_label:
  current_cluster_label_data = result_data[result_data['cluster_label'] == c]
  current_cluster_label_data.reset_index(drop=True, inplace=True)
  users_count += current_cluster_label_data.shape[0]
  if 'index' in current_cluster_label_data.columns:
    current_cluster_label_data = current_cluster_label_data.drop('index', axis=1)
  for pr in current_cluster_label_data.columns[4:]:
    if 1 in current_cluster_label_data[pr].value_counts().index.tolist():
      if pr not in privileges_counts.keys():
        privileges_counts[pr] = 0  
      privileges_counts[pr] += current_cluster_label_data[pr].value_counts()[1]

In [ ]:
users_count

In [ ]:
#privileges_counts = dict(sorted(privileges_counts.items(),key= lambda x:x[1]))
sorted_privileges_counts = sorted(privileges_counts.items(), key=lambda x: x[1], reverse=True)
sorted_privileges_counts

In [ ]:
filtered_privileges_counts = list(filter(lambda x: x[1] > users_count*0.4, sorted_privileges_counts))

#### Filter the most relevant privileges from that cluster

In [ ]:
filtered_privileges_counts

In [ ]:
suggested_privileges = [v[0] for v in filtered_privileges_counts]

In [ ]:
suggested_privileges

### Applying PCA to the model data

To reduce dimentionality of the dataset we are going to use PCA from sckitlearn

In [ ]:
#pca = PCA()

In [ ]:
#new_transformed_privs = pca.fit_transform(user_privs_data.drop('user_id', axis=1))

In [ ]:
#new_transformed_privs.shape

In [ ]:
#transformed_user_privs_data = pd.DataFrame(new_transformed_privs, columns=user_privs_data.drop('user_id', axis=1).columns)

In [ ]:
#transformed_user_privs_data

In [ ]:
#pca.explained_variance_ratio_.shape

In [ ]:
#user_privs_data.drop('user_id', axis=1).columns.shape

In [ ]:
#variance_explained = pd.DataFrame([pca.explained_variance_ratio_], columns=user_privs_data.drop('user_id', axis=1).columns)

In [ ]:
#variance_explained

In [ ]:
#fit_pca_data = agg_clustering.fit(transformed_user_privs_data)

In [ ]:
#fit_pca_data.n_clusters_

In [ ]:
#result = pd.DataFrame(fit_pca_data.labels_, columns=['id'])

In [ ]:
#result

In [ ]:
#result.id.value_counts()